<a href="https://colab.research.google.com/github/Baipzix/BC_Beetle/blob/main/MODIS_TS_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

import pandas as pd
import altair as alt
import numpy as np
import folium


In [3]:
# Trigger the authentication flow.
ee.Authenticate()

In [4]:
# Initialize the library.
ee.Initialize(project='bc-forest-health-v1-0')

In [6]:
# start and end date
date_range = ee.DateRange('2020-01-01', '2023-12-31')

# select the band
datalist = ['LST', 'NDVI', 'EVI']

data_i = 1

 # regional name such as RKB, ROM, RCB, RNO...
rrlist = ['RKB', 'ROM', 'RCB', 'RNO', 'RSK', 'RSC', 'RTO', 'RWC']
rr_i = 2


In [7]:
modislist = ['MODIS/061/MOD11A1', 'MODIS/061/MOD13Q1', 'MODIS/061/MOD13Q1']

modis = modislist[data_i]
modis_band = datalist[data_i]
rr_name = rrlist[rr_i]

resource_region = ee.FeatureCollection('projects/bc-forest-health-v1-0/assets/ResourceRegion')
# Filter the feature collection based on the 'Cl1' attribute
filtered_region = resource_region.filter(ee.Filter.eq('ORG_UNIT', rr_name))

In [9]:
# Function to add Earth Engine image tiles to folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [10]:

aoi = ee.Geometry.Rectangle([-139, 48, -114, 60])  # Replace with actual coordinates

# Load the MODIS/061/MOD11A1 dataset
collection = ee.ImageCollection(modis) \
    .filterDate(date_range) \
    .select(modis_band)


# Clip the LST images to the filtered region
clipped_collection = collection.map(lambda image: image.clip(filtered_region))

# Get the mean LST value for each image in the collection
def compute_mean(image):
    return image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=100  # Resolution in meters
    )

# Map the function over the collection
daily_mean_lst = clipped_collection.mean()


# Create a folium map centered around the AOI
map_center = [55, -125]  # Replace with actual coordinates
mymap = folium.Map(location=map_center, zoom_start=5)

# Add the LST layer to the map
vis_params = {'min': 100, 'max': 10000, 'palette': ['blue', 'green', 'red']}
mymap.add_ee_layer(daily_mean_lst.select(modis_band), vis_params, modis_band)

# Display the map
mymap